## Step 1: Data Preparation & Initial Analysis

In [40]:
import pandas as pd
import numpy as np
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.ensemble import IsolationForest
import holidays

# Load data from an Excel file into a DataFrame
# - parse_dates=['Date']: Ensures that the 'Date' column is parsed as datetime objects
# - engine='openpyxl': Specifies the engine to use for reading the Excel file
df = pd.read_excel('../data/raw/Hotel_Revenue_Data.xlsx', parse_dates=['Date'], engine='openpyxl')

# Create temporal features directly in the main DataFrame
# Extract year, month, and day of the week from the 'Date' column
df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month
df['DayOfWeek'] = df['Date'].dt.weekday  # Monday=0, Sunday=6
# Determine if the day is a weekend in the UAE (Friday and Saturday)
df['IsWeekend'] = df['Date'].dt.weekday >= 4

# Define all revenue centers as a list of strings
# This creates a list of revenue center names from 'RevenueCenter_1' to 'RevenueCenter_9'
all_revenue_centers = [f'RevenueCenter_{i}' for i in range(1, 10)]

# Create a complete index of all possible combinations of dates, meal periods, and revenue centers
# - date_range: Unique dates from the DataFrame
# - meal_periods: Unique meal periods from the DataFrame
date_range = df['Date'].unique()
meal_periods = df['MealPeriod'].unique()


# Create a multi-index that represents every possible combination of:
# - date_range: All unique dates in the dataset
# - meal_periods: All meal periods (Breakfast, Lunch, Dinner)
# - all_revenue_centers: All revenue centers (RevenueCenter_1 through RevenueCenter_9)
# This ensures we have a complete grid of all possible combinations, even if some don't exist in the data
# The resulting index will be used to identify missing data points and fill them with zeros
full_index = pd.MultiIndex.from_product(
    [date_range, meal_periods, all_revenue_centers],
    names=['Date', 'MealPeriod', 'RevenueCenterName']
)

# Create a base DataFrame with only the essential columns for merging
base_df = df[['Date', 'MealPeriod', 'RevenueCenterName', 'CheckTotal']]

# Create the complete DataFrame using merge instead of reindex
# - Reset the index of the DataFrame created from the multi-index
# - Merge with base_df to align data based on 'Date', 'MealPeriod', and 'RevenueCenterName'
# - how='left': Ensures all combinations in full_index are retained, filling missing data with NaN
df_complete = pd.DataFrame(index=full_index).reset_index()
df_complete = pd.merge(
    df_complete,
    base_df,
    on=['Date', 'MealPeriod', 'RevenueCenterName'],
    how='left'
)

# Fill missing values in 'CheckTotal' with 0
# This assumes that missing values in 'CheckTotal' imply zero revenue
df_complete['CheckTotal'] = df_complete['CheckTotal'].fillna(0)

# Add temporal features from unique dates to the complete DataFrame
# - drop_duplicates(): Ensures each date is unique in the date_features DataFrame
df_complete = pd.merge(
    df_complete,
    df[['Date', 'Year', 'Month', 'DayOfWeek', 'IsWeekend']].drop_duplicates(),
    on='Date',
    how='left'
)

# Calculate daily totals by grouping the complete DataFrame
# - Group by 'Date', 'MealPeriod', 'RevenueCenterName', 'DayOfWeek', 'Month', 'Year'
# - Sum the 'CheckTotal' for each group to get daily totals
df_totals = df_complete.groupby(
    ['Date', 'MealPeriod', 'RevenueCenterName', 'DayOfWeek', 'Month', 'Year'],
    as_index=False
)['CheckTotal'].sum()

# Print the first 20 rows of the resulting DataFrame to verify the results
df_totals.head(20)

,Date,MealPeriod,RevenueCenterName,DayOfWeek,Month,Year,CheckTotal
0,2023-01-01,BreakFast,RevenueCenter_1,6,1,2023,1499.4
1,2023-01-01,BreakFast,RevenueCenter_2,6,1,2023,35.0
2,2023-01-01,BreakFast,RevenueCenter_3,6,1,2023,0.0
3,2023-01-01,BreakFast,RevenueCenter_4,6,1,2023,0.0
4,2023-01-01,BreakFast,RevenueCenter_5,6,1,2023,21807.0
5,2023-01-01,BreakFast,RevenueCenter_6,6,1,2023,93.0
6,2023-01-01,BreakFast,RevenueCenter_7,6,1,2023,0.0
7,2023-01-01,BreakFast,RevenueCenter_8,6,1,2023,0.0
8,2023-01-01,BreakFast,RevenueCenter_9,6,1,2023,0.0
9,2023-01-01,Dinner,RevenueCenter_1,6,1,2023,4374.5


## Step 2: Zero Value Identification

In [38]:
def analyze_zeros(df_totals):
    # --- Data Preparation ---
    print("\n" + "="*50)
    print("1. Normalizing meal period names...")
    # Standardize meal period naming
    # Standardize meal period names by stripping whitespace and capitalizing each word
    df_totals['MealPeriod'] = df_totals['MealPeriod'].str.strip().str.title()
    print(f"Unique meal periods: {df_totals['MealPeriod'].unique()}")
    
    # --- Data Validation ---
    print("\n" + "="*50)
    print("2. Data integrity checks...")
    required_columns = ['CheckTotal', 'RevenueCenterName', 'MealPeriod', 'Date']
    missing_cols = [col for col in required_columns if col not in df_totals.columns]
    if missing_cols:
        raise KeyError(f"Missing critical columns: {missing_cols}")
    print("✓ All required columns present")
    
    # --- Temporal Analysis ---
    print("\n" + "="*50)
    print("3. Performing temporal analysis...")
    
    # Group the DataFrame by 'RevenueCenterName', 'MealPeriod', and 'Date'
    # For each group, calculate two metrics:
    # 1. zero_count: Count the number of entries where 'CheckTotal' is zero.
    # 2. total_days: Count the total number of entries (days) in each group.
    # Reset the index to flatten the DataFrame after aggregation.
    temporal_analysis = df_totals.groupby(['RevenueCenterName', 'MealPeriod', 'Date'])\
        .agg(zero_count=('CheckTotal', lambda x: sum(x == 0)),
             total_days=('CheckTotal', 'count')).reset_index()
    
    # --- Pivot Table Creation ---
    print("\n" + "="*50)
    print("4. Creating heatmap pivot table...")
    # Ensure all meal periods are represented
    expected_meals = ['Breakfast', 'Lunch', 'Dinner']
    pivot_table = pd.pivot_table(
        temporal_analysis,
        values='zero_count',
        index=['RevenueCenterName'],
        columns=['MealPeriod'],
        aggfunc='mean'
    ).reindex(columns=expected_meals, fill_value=0)
    
    # --- Safe Observations ---
    print("\n" + "="*50)
    print("5. Key Observations:")
    # Breakfast analysis
    if pivot_table['Breakfast'].sum() > 0:
        print(f"Highest breakfast zeros: {pivot_table['Breakfast'].idxmax()} ({pivot_table['Breakfast'].max():.2f})")
    else:
        print("⚠️ No breakfast data available")
    
    # Dinner analysis
    dinner_data = pivot_table.get('Dinner', pd.Series())
    if not dinner_data.empty and dinner_data.sum() > 0:
        print(f"Most consistent dinner: {dinner_data.idxmin()} ({dinner_data.min():.2f})")
    else:
        print("⚠️ No dinner data available")
    
    return temporal_analysis, pivot_table

# --- Execution with Enhanced Logging ---
print("\n" + "="*50)
print("INITIALIZING ANALYSIS PIPELINE")
print("Loading dataset...")
print(f"Initial columns: {df_totals.columns.tolist()}")
print(f"Date range: {df_totals['Date'].min().date()} to {df_totals['Date'].max().date()}")

try:
    zero_analysis, zero_heatmap = analyze_zeros(df_totals)
    
    print("\n" + "="*50)
    print("FINAL RESULTS:")
    print("Temporal Analysis Preview:")
    print(zero_analysis.sample(3))
    print("\nHeatmap Summary:")
    print(zero_heatmap.describe())
    
except Exception as e:
    print("\n" + "="*50)
    print("❌ ANALYSIS FAILED")
    print(f"Error: {str(e)}")
    print("Troubleshooting Tips:")
    print("- Verify meal period values in source data")
    print("- Check for missing columns in input CSV")
    print("- Ensure dates are properly formatted")


INITIALIZING ANALYSIS PIPELINE
Loading dataset...
Initial columns: ['Date', 'MealPeriod', 'RevenueCenterName', 'DayOfWeek', 'Month', 'Year', 'CheckTotal']
Date range: 2023-01-01 to 2024-04-30

1. Normalizing meal period names...
Unique meal periods: ['Breakfast' 'Dinner' 'Lunch']

2. Data integrity checks...
✓ All required columns present

3. Performing temporal analysis...

4. Creating heatmap pivot table...

5. Key Observations:
Highest breakfast zeros: RevenueCenter_4 (1.00)
Most consistent dinner: RevenueCenter_1 (0.00)

FINAL RESULTS:
Temporal Analysis Preview:
     RevenueCenterName MealPeriod       Date  zero_count  total_days
1650   RevenueCenter_2  Breakfast 2023-07-12           0           1
8515   RevenueCenter_6      Lunch 2023-09-11           0           1
2792   RevenueCenter_2      Lunch 2023-12-29           0           1

Heatmap Summary:
MealPeriod  Breakfast     Lunch    Dinner
count        9.000000  9.000000  9.000000
mean         0.381344  0.286465  0.271376
std   